In [2]:
from diffusion.utils import *
import d3rlpy
from d3rlpy.algos import SAC, SACConfig
from sklearn.model_selection import train_test_split

base_agent_data_path = '/home/anhquanpham/projects/data'
base_synthetic_data_path = '/home/anhquanpham/projects/results/non_comp_diff_124'

base_results_folder = '/home/anhquanpham/projects/results/d3rlpy'

In [3]:
robot = 'Jaco'
obj = 'Plate'
obst = 'GoalWall'
subtask = 'PickPlace'

env_name = f"{robot}_{obj}_{obst}_{subtask}"

data_type = 'agent'

if data_type == 'synthetic':
    synthetic_run_id = 20
    mode = 'train'

if data_type == 'agent':
    env = composuite.make(robot, obj, obst, subtask, use_task_id_obs=True, ignore_done=False)
    dataset = load_single_composuite_dataset(base_path=base_agent_data_path, 
                                             dataset_type='expert', 
                                             robot=robot, obj=obj, 
                                             obst=obst, task=subtask)
    dataset, _ = remove_indicator_vectors(env.modality_dims, transitions_dataset(dataset))

if data_type == 'synthetic':
    dataset = load_single_synthetic_dataset(base_path=os.path.join(base_synthetic_data_path, synthetic_run_id, mode), 
                                            robot=robot, obj=obj, 
                                            obst=obst, task=subtask)

/home/anhquanpham/.pyenv/versions/first_3.9.6/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
# print(dataset['observations'].shape)
# print(env.reset())

In [5]:
dataset = d3rlpy.dataset.MDPDataset(
    observations=dataset['observations'],
    actions=dataset['actions'],
    rewards=dataset['rewards'],
    terminals=dataset['terminals']
)

2025-04-11 21:31.27 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(8,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(77,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-04-11 21:31.27 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2025-04-11 21:31.27 [info     ] Action size has been automatically determined. action_size=8


In [6]:
# sac = d3rlpy.algos.SAC(scaler='standard', action_scaler='min_max')
# sac.build_with_dataset(dataset)

####WHAT TO FILL IN HERE?#####

sac = d3rlpy.algos.SAC(SACConfig(), device="cpu", enable_ddp=False)
sac.build_with_dataset(dataset)


In [7]:
# print(type(dataset))
# print(dir(dataset))

In [8]:
# print(type(dataset.episodes))
# print(dir(dataset.episodes))

In [9]:
episode_lengths = [len(ep) for ep in dataset.episodes]
# print("Episode lengths:", episode_lengths)

In [10]:
#print(dataset.episodes[0])

In [11]:
# train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)
# eval_env = composuite.make(robot, obj, obst, subtask, ignore_done=False)

train_episodes, test_episodes = train_test_split(dataset.episodes, test_size=0.2)
eval_env = composuite.make(robot, obj, obst, subtask, ignore_done=False)

# # For training data:
# train_dataset = d3rlpy.dataset.MDPDataset.from_episode_generator(iter(train_episodes))
# # For testing data:
# test_dataset = d3rlpy.dataset.MDPDataset.from_episode_generator(iter(test_episodes))

# train_dataset = d3rlpy.dataset.MDPDataset.from_episode_generator(
#     lambda: iter(train_episodes),
#     buffer=dataset.buffer
# )
# test_dataset  = d3rlpy.dataset.MDPDataset.from_episode_generator(
#     lambda: iter(test_episodes),
#     buffer=dataset.buffer
# )


/home/anhquanpham/.pyenv/versions/first_3.9.6/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [12]:
#print("Train", train_episodes[0])

In [13]:
#print("Eval_env", type(eval_env.reset()))

In [14]:
# print("Eval_env", eval_env.reset().shape)

# Wrap the environment with the ResetAdapter.
# eval_env = ResetAdapter(eval_env)

# print("Eval_env_1", eval_env.reset())
# print("Observation shape (should be 77):", observation.shape)
# print("Condition shape (should be 16):", cond.shape)

# Now, pass this wrapped environment to your evaluator.
from d3rlpy.metrics import EnvironmentEvaluator

env_evaluator = EnvironmentEvaluator(env=eval_env, n_trials=10, epsilon=0.0)
score = env_evaluator(sac, dataset)
#print("Evaluation score:", score)

In [15]:
# evaluate_scorer = d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
# rewards = evaluate_scorer(sac)

In [16]:
import numpy as np
from d3rlpy.dataset import MDPDataset

# Initialize empty lists to collect transitions
obs_list, act_list, rew_list, term_list = [], [], [], []
for episode in train_episodes:
    # Assuming each episode is an object with these attributes
    obs_list.append(episode.observations)
    act_list.append(episode.actions)
    rew_list.append(episode.rewards)
    term_list.append(np.atleast_1d(episode.terminated))

# Concatenate along the first axis (number of samples)
observations = np.concatenate(obs_list, axis=0)
actions = np.concatenate(act_list, axis=0)
rewards = np.concatenate(rew_list, axis=0)
print(len(term_list))
terminals = np.concatenate(term_list, axis=0)

# Create an MDPDataset (which is an instance of ReplayBufferBase)
train_dataset = MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals
)


1599
2025-04-11 21:31.32 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(8,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(77,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-04-11 21:31.32 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2025-04-11 21:31.32 [info     ] Action size has been automatically determined. action_size=8


In [ ]:
# sac.fit(train_episodes,
#         eval_episodes=test_episodes,
#         n_epochs=10,
#         save_interval=10,
#         scorers={
#             "environment": env_evaluator 
#             }
#         )

# After building your dataset
print("Number of episodes:", len(dataset.episodes))

# Pick the first episode (or any episode you want to inspect)
episode = dataset.episodes[0]
episode_reward = sum(episode.rewards)
print("Total return for one episode:", episode_reward)

sac.fit(
    dataset=train_dataset,         # Your ReplayBufferBase (MDPDataset) instance
    n_steps=1000000,                # Total training steps (increased to 100k)
    n_steps_per_epoch=500,        # 5000 steps per epoch (results in 20 epochs total)
    save_interval=20000,           # Save every 20k steps (every 4 epochs)
    evaluators={"environment": env_evaluator}  # Your evaluator dictionary
)

Number of episodes: 1999
Total return for one episode: [336.89246]
2025-04-11 21:31.32 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(77,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(8,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=8)
2025-04-11 21:31.32 [warning  ] Skip building models since they're already built.
2025-04-11 21:31.32 [info     ] Directory is created at d3rlpy_logs/SAC_20250411213132
2025-04-11 21:31.32 [info     ] Parameters                     params={'observation_shape': [77], 'action_size': 8, 'config': {'type': 'sac', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0003, 'critic_learnin

Epoch 1/2000:   0%|          | 0/500 [00:00<?, ?it/s]

2025-04-11 21:31.45 [info     ] SAC_20250411213132: epoch=1 step=500 epoch=1 metrics={'time_sample_batch': 0.006802546501159668, 'time_algorithm_update': 0.014700082778930664, 'critic_loss': 0.023832643565023317, 'actor_loss': -5.681400044441223, 'temp': 0.9298567875623703, 'temp_loss': 12.52301689529419, 'time_step': 0.02159464406967163, 'environment': 0.00045003382760134826} step=500


Epoch 2/2000:   0%|          | 0/500 [00:00<?, ?it/s]

2025-04-11 21:32.01 [info     ] SAC_20250411213132: epoch=2 step=1000 epoch=2 metrics={'time_sample_batch': 0.00794410753250122, 'time_algorithm_update': 0.01767028570175171, 'critic_loss': 0.000706411643885076, 'actor_loss': -5.029967071533203, 'temp': 0.807664516210556, 'temp_loss': 10.88017645072937, 'time_step': 0.0257167272567749, 'environment': 0.00043670446578652827} step=1000


...

Epoch 68/2000:   0%|          | 0/500 [00:00<?, ?it/s]

2025-04-11 21:50.25 [info     ] SAC_20250411213132: epoch=68 step=34000 epoch=68 metrics={'time_sample_batch': 0.007968637466430664, 'time_algorithm_update': 0.02357327938079834, 'critic_loss': 8.220662562280267e-06, 'actor_loss': -0.6501296644210816, 'temp': 0.0007979723971802741, 'temp_loss': 0.0008647980083478614, 'time_step': 0.03164574480056763, 'environment': 0.0004586901769488506} step=34000


Epoch 69/2000:   0%|          | 0/500 [00:00<?, ?it/s]

2025-04-11 21:50.46 [info     ] SAC_20250411213132: epoch=69 step=34500 epoch=69 metrics={'time_sample_batch': 0.008504099369049072, 'time_algorithm_update': 0.029692021369934082, 'critic_loss': 6.502890619117352e-06, 'actor_loss': -0.6501362017393112, 'temp': 0.0007630237818229943, 'temp_loss': 0.0006028518888560938, 'time_step': 0.03831096076965332, 'environment': 0.0004097273877026941} step=34500


Epoch 70/2000:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("d3rlpy_logs/SAC_20250411210953/environment.csv")

df.columns = ["epoch", "total_step", "env_return"]


plt.plot(df["total_step"], df["env_return"], label="Environment Return")
plt.xlabel("Total Steps")
plt.ylabel("Return")
plt.legend()
plt.show()

In [ ]:
print(df.columns)